# Análise da Correlação de Métricas de Países com sua Emissão de Carbono

## Alunos

- Bruno de Marco Appolonio - RA
- João Vitor Vendemiato Fatoretto - 199944
- Lucas Costa de Oliveira - RA
- Natan Beltrão da Cunha Pevidor Carvalho - 184972

## Introdução

Neste projeto, buscamos analisar a correlação entre diversas métricas e a emissão de carbono anual de um país. As métricas avaliadas incluem tanto fatores diretamente relacionados com a pegada de carbono de um país (como desmatamento) tanto como métricas mais gerais (como liberdade de imprensa), permitindo estabelecer relações diversas.

Como metodologia, aplicamos uma regressão linear sobre os diversos parâmetros, permitindo avaliar a relevância de cada um para o cálculo e criando um modelo de predição considerando mudanças nas métricas avaliadas.

## Métricas Utilizadas

Obtivemos as métricas de diversas fontes. Por isso, nem todos os dados estão disponíveis para todos os países avaliados ou para o mesmo ano. Utilizamos o dados mais recentes disponíveis em cada uma das fontes.

### Our World in Data ([fonte](https://ourworldindata.org/co2-emissions))

- Emissão de CO<sub>2</sub> em milhões de toneladas (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- PIB em dólares ajustdo para inflação de 2011 (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)
- População (2015 para Cuba, Coreia do Norte e Palestina, 2016 par países restantes)

### ONU ([fonte](http://hdr.undp.org/en/indicators))

- **Índice de gini de distribuição de renda familiar (anos variados)**
- IDH (2019)
- Porcentagem da população residindo em áreas urbanas (2019)
- Porcentagem da variação na área de floresta (diferença entre 1990 e 2016)
- **Porcentagem da energia consumida proveniente de combustíveis fósseis (anos variados)**
- Porcentagem do Rendimento Nacional Bruto derivado da extração de recursos naturais (anos variados)
- Índice de desigualdade de gênero (2019)
- Expectativa de vida no nascimento (2019)
- **Porcentagem da população vivendo abaixo da linha da pobreza (anos variados)**
- **Porcentagem do PIB investido em pesquisa e desenvolvimento (anos variados)**
- **Proporção dos gastos públicos em educação e saúde sobre gastos militares (anos variados)**
- Porcentagem de importações e exportações sobre o PIB (anos variados)
- **Índice de desemprego (2019)**

### Center for Systemic Peace ([fonte](https://www.systemicpeace.org/polityproject.html))

- Polity Score, índice que avalia o nível de democracia de cada país - removemos países com valores especiais para evitar interferência na regressão (2018)

### Repórteres sem Fronteira ([fonte](https://rsf.org/en/ranking))

- Índice de liberdade de imprensa (2021)

**Dados em negrito estão disponíveis para menos de 90% dos países selecionados**

In [2]:
import json
import csv

data = {}

co2_file_path = "owid-co2-data.json"
with open(co2_file_path, 'r') as json_file:
    co2_data = json.load(json_file)
    for key in co2_data.keys():
        if 'iso_code' in co2_data[key] and key != 'World':
            for i in range(len(co2_data[key]['data'])):
                if 'co2' in co2_data[key]['data'][i] and 'gdp' in co2_data[key]['data'][i] \
                        and 'population' in co2_data[key]['data'][i]:
                    data[co2_data[key]['iso_code']] = {
                        'name': key,
                        'co2_emissions': co2_data[key]['data'][i]['co2'],
                        'gdp': co2_data[key]['data'][i]['gdp'],
                        'population': co2_data[key]['data'][i]['population']               
                    }

X_fields = ['gdp', 'population']
                   
def get_iso3_from_country_name(country_name):
    iso3 = [key for key, values in data.items() if values['name'] == country_name]
    return iso3[0] if iso3 else None

def add_csv_data(file, country_row, data_row, data_name):
    X_fields.append(data_name)
    with open(file) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [(data[key]['name'], key) for key in data]
        for line in reader:
            country_key = get_iso3_from_country_name(line[country_row])
            if country_key:
                data[country_key][data_name] = float(line[data_row])

def check_missing_countries(file, country_row):
    with open(file) as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        co2_names = [data_dict[key]['name'] for key in data_dict]
        file_names = [line[country_row] for line in reader]
        return [name for name in co2_names if name not in file_names]

add_csv_data('gini-index.csv', 1, -2, 'gini_index')
add_csv_data('hdi.csv', 1, -2, 'hdi')
add_csv_data('urban-population.csv', 1, -2, 'urban_population')
add_csv_data('forest-area-change.csv', 1, -2, 'forest_area_change')
add_csv_data('fossil-fuel-percentage.csv', 1, -2, 'fossil_fuel_percentage')
add_csv_data('natural-resource-depletion.csv', 1, -2, 'natural_resource_depletion')
add_csv_data('gender-inequality-index.csv', 1, -2, 'gender_inequality_index')
add_csv_data('life-expectancy.csv', 1, -2, 'life_expectancy')
add_csv_data('poverty-line.csv', 1, -2, 'poverty_line')
add_csv_data('research-and-development.csv', 1, -2, 'research_and_development')
add_csv_data('education-health-military-expenses.csv', 1, -2, 'education_health_military_expenses')
add_csv_data('exports-imports.csv', 1, -2, 'exports_imports')
add_csv_data('unemployment.csv', 1, -2, 'unemployment')
add_csv_data('democracy.csv', 4, 10, 'democracy')
add_csv_data('press-freedom.csv', 3, 7, 'press_freedom')

print('Países:', len(data.keys()))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 229: character maps to <undefined>

## Pré-processamento dos Dados

Antes de executar a regressão, precisamos realizar algumas alterações nos dados para garantir um melhor resultado.

### Imputação de Dados Faltantes

Devido ao uso de diversas fontes de dados diferentes, muitas métricas não estão disponíveis para todos os países. Porém, o método de regressão linear não é capaz de lidar com valores nulos. Por isso, utilizamos um método simples de imputação de dados, onde valores nulos passam a ter o valor da média aritmética dos valores restantes. Essa atribuição tenta atribuir o valor que tenha o menor impacto possível no treinamento.

### Separação de Dados de Treinamento e Teste

Para validar nosso modelo, reservamos 10% dos países para teste no final e 90% para o treinamento e validações necessárias durante seu desenvolvimento.


### Normalização dos Dados

In [3]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

countries = []
X = []
y = []
for values in data.values():
    countries.append(values['name'])
    y.append(values['co2_emissions'])
    X.append([values[field] if field in values else np.nan for field in X_fields])

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp.fit_transform(X)
y = np.array(y)

X_train, X_test, y_train, y_test, countries_train, countries_test \
        = train_test_split(X, y, countries, test_size=0.1, random_state=0)

print("Training countries:", countries_train)
print("Testing countries:", countries_test)

Training countries: ['Niger', 'Myanmar', 'Zambia', 'North Macedonia', 'Sudan', 'Costa Rica', 'Germany', 'Togo', 'Azerbaijan', 'Turkmenistan', 'Georgia', 'Tanzania', 'Sao Tome and Principe', 'Guinea-Bissau', 'Haiti', 'United Kingdom', 'Canada', 'Taiwan', 'Slovakia', 'Bulgaria', 'Mexico', 'South Africa', 'Bosnia and Herzegovina', 'Honduras', 'North Korea', 'Djibouti', 'India', 'Cambodia', 'Yemen', 'Japan', 'Rwanda', 'Laos', 'Saint Lucia', 'Chad', 'Montenegro', 'Russia', 'Tunisia', 'Hong Kong', 'Nigeria', 'Brazil', 'Israel', 'Colombia', 'Libya', 'Jamaica', 'Oman', 'Algeria', 'Guinea', 'Lithuania', 'Turkey', 'Ecuador', 'Bangladesh', 'Malta', 'Uzbekistan', 'Mozambique', 'Mauritania', 'Lesotho', 'Sierra Leone', 'Uruguay', 'Finland', 'Portugal', 'Trinidad and Tobago', 'Hungary', 'Norway', 'Pakistan', 'Iraq', 'Ethiopia', 'Eswatini', 'Belgium', 'Romania', 'Cameroon', 'Philippines', 'Burkina Faso', 'Bolivia', 'Kuwait', 'Gabon', 'Kazakhstan', 'Angola', 'Zimbabwe', 'Mongolia', 'Australia', 'Iran',

In [10]:
#Regressao linear normal
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import pandas as pd
#Utilizando a mesma regressão logistica com liblinear pois é o recomendado para datasets pequenos(<10000)
reg = LinearRegression().fit(X_train, y_train)
print(reg.score(X_train, y_train))
print("Intercept")
print(reg.intercept_)
print("Coeficiente")
print(reg.coef_)
y_pred = reg.predict(X_test)
errors = mean_squared_error(y_test, y_pred)
print("Erro pela media quadratica")
print(errors)
errors2 = mean_squared_error(y_test, y_pred, squared=False)
print("Erro pela raiz da media quadratica")
print(errors2)
errors3 = mean_absolute_error(y_test, y_pred)
print("Erro pela media absoluta")
print(errors3)
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df

0.922896410881841
Intercept
473.08026533298175
Coeficiente
[ 3.48237233e-10  1.13110013e-06  2.76726980e+00 -9.97225549e+02
  3.49413939e-01  3.23578043e-01 -1.52925772e-01  2.00143280e+00
 -6.49600814e+02  3.65369924e+00 -2.11518868e+00 -1.55057499e+01
  3.45723429e+00  4.09032556e-01  4.77446003e+00 -1.18249108e+01]
Erro pela media quadratica
31752.640821238947
Erro pela raiz da media quadratica
178.1927069810629
Erro pela media absoluta
127.14981518149543


,Actual,Predicted
0,281.705,363.327189
1,1.263,-108.361145
2,5.448,27.285985
3,67.112,20.675322
4,4.346,-14.165155
5,16.547,-44.787789
6,617.960,522.785808
7,6.638,-51.718853
8,0.297,-176.261503
9,233.516,466.246464
